In [1]:
# coding:UTF-8
# importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
% matplotlib inline

import math

In [2]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.

    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    # defining a blank mask to start with
    mask = np.zeros_like(img)
    # defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    # filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    # returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


In [3]:
def draw_lines(img,lines,color,thickness):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img,(x1,y1),(x2,y2),color,thickness)

In [4]:
# 霍夫变换函数
def hough_lines(img):
    """
    `img` should be the output of a Canny transform.

    Returns an image with hough lines drawn.
    """
    rho = 2
    theta = np.pi/180
    threshold = 15
    min_line_len = 60
    max_line_gap = 30
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len,
                            maxLineGap=max_line_gap)

    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, [255,0,0], 2)
    cv2.imshow('hough_image',line_img)
    return lines,line_img

In [5]:
def Lane_finding(image):
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_blur = cv2.GaussianBlur(image_gray, (5,5), 0)
    low_threshold = 50
    high_threshold = 150
    image_canny = cv2.Canny(image_blur, low_threshold, high_threshold)
    cv2.imshow('image_canny', image_canny)
    # 图像像素行数
    rows = image_canny.shape[0]  # 540行
    # 图像像素列数
    cols = image_canny.shape[1]  # 960列
    left_bottom = [0, rows]  # [0,540]
    right_bottom = [cols, rows]  # [960,540]
    apex = [cols / 2, rows*0.6]  # [480,310]
    vertices = np.array([[left_bottom, right_bottom, apex]], np.int32)
    roi_image = region_of_interest(image_canny, vertices)
    cv2.imshow('roi_image', roi_image)
    lines,hough_image = hough_lines(roi_image)
    # 将得到线段绘制在原始图像上
    line_image = np.copy(image)
    draw_lines(line_image, lines, [255, 0, 0], 2)
    # line_image = cv2.addWeighted(line_image, 0.8, hough_image, 1, 0)
    cv2.imshow('src', line_image)
    cv2.waitKey(0)
    return hough_image

In [6]:
image = mpimg.imread("/Users/zhangzheng/Desktop/lane_lines_finding/test_images/solidYellowLeft.jpg")

In [ ]:
if __name__ == '__main__':
    Lane_finding(image)